In [42]:
# Install required packages
!pip install ipywidgets numpy matplotlib scipy --quiet

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- CONSTANTS ---
G = 6.67430e-11          # Gravitational constant (m^3 kg^-1 s^-2)
R_EARTH = 6371e3         # Earth radius (m)
M_EARTH = 5.972e24       # Earth mass (kg)
J2 = 1.08263e-3          # Earth's J2 coefficient (dimensionless)
R_MOON = 1737.4e3        # Moon radius (m)
M_MOON = 7.342e22        # Moon mass (kg)
MOON_DIST = 384400e3     # Mean Earth-Moon distance (m)
MOON_PERIOD = 27.321661 * 86400  # Moon sidereal period (s)

def atmospheric_density(altitude):
    if altitude < 0:
        return 0.0
    rho0 = 1.225  # kg/m^3 at sea level
    H = 8500      # scale height (m)
    return rho0 * np.exp(-altitude / H)

def atmospheric_drag(v, rho, Cd, A, m):
    return -0.5 * rho * Cd * A * np.linalg.norm(v) * v / m

def j2_perturbation(r):
    r_norm = np.linalg.norm(r)
    x, y, z = r
    factor = 1.5 * J2 * G * M_EARTH * R_EARTH**2 / r_norm**5
    z2 = z*z
    r2 = r_norm*r_norm
    ax = factor * x * (5*z2/r2 - 1)
    ay = factor * y * (5*z2/r2 - 1)
    az = factor * z * (5*z2/r2 - 3)
    return np.array([ax, ay, az])

def moon_position(t):
    theta = 2 * np.pi * t / MOON_PERIOD
    return MOON_DIST * np.array([np.cos(theta), np.sin(theta), 0.0])

def moon_gravity(r_sat, t):
    r_moon = moon_position(t)
    r = r_sat - r_moon
    r_mag = np.linalg.norm(r)
    if r_mag == 0:
        return np.zeros(3)
    return -G * M_MOON * r / r_mag**3

def two_body_eq(t, y, m_sat, Cd, A, include_drag, include_j2, include_moon):
    r = y[0:3]
    v = y[3:6]
    r_norm = np.linalg.norm(r)
    acc_gravity = -G * M_EARTH * r / r_norm**3
    acc_j2 = j2_perturbation(r) if include_j2 else np.zeros(3)
    acc_drag = np.zeros(3)
    if include_drag:
        alt = r_norm - R_EARTH
        rho = atmospheric_density(alt)
        acc_drag = atmospheric_drag(v, rho, Cd, A, m_sat)
    acc_moon = moon_gravity(r, t) if include_moon else np.zeros(3)
    acc = acc_gravity + acc_j2 + acc_drag + acc_moon
    return np.concatenate((v, acc))

def run_simulation(m_sat, r0, v0, t_span, t_eval, Cd, A, include_drag, include_j2, include_moon):
    y0 = np.concatenate((r0, v0))
    sol = solve_ivp(
        two_body_eq, t_span, y0,
        args=(m_sat, Cd, A, include_drag, include_j2, include_moon),
        method='DOP853', rtol=1e-12, atol=1e-12, t_eval=t_eval
    )
    return sol

def run_simulation_interface():
    header = widgets.HTML('''
    <div style="background: #0B3D91; color: white; padding: 10px; border-radius: 5px;">
        <h1 style="margin:0; text-align:center;">NASA Orbital Dynamics Lab</h1>
        <p style="text-align:center; font-size:16px;">
            <b>Advanced Orbital Mechanics Simulator</b><br>
            Physics: Newtonian gravity, J2, drag, Moon gravity
        </p>
    </div>
    ''')

    mass_sat = widgets.FloatText(value=1000.0, description='Mass Sat (kg):')
    r_x = widgets.FloatText(value=R_EARTH/1000 + 500, description='Sat X (km):')
    r_y = widgets.FloatText(value=0.0, description='Sat Y (km):')
    r_z = widgets.FloatText(value=0.0, description='Sat Z (km):')
    v_x = widgets.FloatText(value=0.0, description='Sat Vx (km/s):')
    v_y = widgets.FloatText(value=7.6, description='Sat Vy (km/s):')
    v_z = widgets.FloatText(value=0.0, description='Sat Vz (km/s):')
    sim_days = widgets.IntSlider(value=1, min=1, max=30, step=1, description='Days:')
    drag_toggle = widgets.Checkbox(value=False, description='Atmospheric Drag')
    j2_toggle = widgets.Checkbox(value=True, description='J2 Perturbation')
    moon_toggle = widgets.Checkbox(value=False, description='Moon Gravity')
    run_btn = widgets.Button(description="Run Simulation", button_style='success')
    out = widgets.Output()

    def on_click(b):
        with out:
            clear_output()
            print("Running simulation... (may take a few seconds for longer runs)")
            t_total = sim_days.value * 86400
            t_eval = np.linspace(0, t_total, 2000)
            r0 = np.array([r_x.value*1000, r_y.value*1000, r_z.value*1000])
            v0 = np.array([v_x.value*1000, v_y.value*1000, v_z.value*1000])

            sol = run_simulation(
                m_sat=mass_sat.value,
                r0=r0,
                v0=v0,
                t_span=(0, t_total),
                t_eval=t_eval,
                Cd=2.2,
                A=4.0,
                include_drag=drag_toggle.value,
                include_j2=j2_toggle.value,
                include_moon=moon_toggle.value
            )

            r_path = sol.y[0:3].T / 1000  # km
            v_path = sol.y[3:6].T / 1000  # km/s
            r_mag = np.linalg.norm(r_path, axis=1)
            v_mag = np.linalg.norm(v_path, axis=1)

            eccentricity = []
            angular_momentum = []
            for r, v in zip(sol.y[0:3].T, sol.y[3:6].T):
                h = np.cross(r, v)
                e_vec = np.cross(v, h)/(G*M_EARTH) - r/np.linalg.norm(r)
                eccentricity.append(np.linalg.norm(e_vec))
                angular_momentum.append(np.linalg.norm(h))

            kinetic_energy = 0.5 * v_mag**2 * 1e6
            potential_energy = -G * M_EARTH / (r_mag*1e3)
            total_energy = kinetic_energy + potential_energy
            t_hours = sol.t / 3600

            moon_path = np.array([moon_position(t) for t in sol.t]) / 1000  # km

            fig = plt.figure(figsize=(20, 14))
            gs = fig.add_gridspec(3, 3)

            ax1 = fig.add_subplot(gs[:2,0:2], projection='3d')
            ax1.plot(r_path[:,0], r_path[:,1], r_path[:,2], label='Satellite Path', color='orangered')
            u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
            x = (R_EARTH/1000)*np.cos(u)*np.sin(v)
            y = (R_EARTH/1000)*np.sin(u)*np.sin(v)
            z = (R_EARTH/1000)*np.cos(v)
            ax1.plot_surface(x, y, z, color='b', alpha=0.3)
            if moon_toggle.value:
                ax1.plot(moon_path[:,0], moon_path[:,1], moon_path[:,2], color='gray', label='Moon Orbit')
                moon_x, moon_y, moon_z = moon_path[-1]
                u2, v2 = np.mgrid[0:2*np.pi:10j, 0:np.pi:5j]
                mx = (R_MOON/1000)*np.cos(u2)*np.sin(v2) + moon_x
                my = (R_MOON/1000)*np.sin(u2)*np.sin(v2) + moon_y
                mz = (R_MOON/1000)*np.cos(v2) + moon_z
                ax1.plot_surface(mx, my, mz, color='gray', alpha=0.5)
            ax1.set_title('3D Orbit Path')
            ax1.set_xlabel('X (km)')
            ax1.set_ylabel('Y (km)')
            ax1.set_zlabel('Z (km)')
            ax1.legend()

            ax2 = fig.add_subplot(gs[0,2])
            ax2.plot(t_hours, r_mag, color='purple')
            ax2.set_title('Distance from Earth Over Time')
            ax2.set_xlabel('Time (hours)')
            ax2.set_ylabel('Distance (km)')

            ax3 = fig.add_subplot(gs[1,2])
            ax3.plot(t_hours, v_mag, color='green')
            ax3.set_title('Velocity Over Time')
            ax3.set_xlabel('Time (hours)')
            ax3.set_ylabel('Velocity (km/s)')

            ax4 = fig.add_subplot(gs[2,0])
            ax4.plot(t_hours, eccentricity, color='darkred')
            ax4.set_title('Eccentricity Over Time')
            ax4.set_xlabel('Time (hours)')
            ax4.set_ylabel('Eccentricity')

            ax5 = fig.add_subplot(gs[2,1])
            ax5.plot(t_hours, kinetic_energy, label='Kinetic', color='blue')
            ax5.plot(t_hours, potential_energy, label='Potential', color='orange')
            ax5.plot(t_hours, total_energy, label='Total', color='black')
            ax5.set_title('Specific Energy Over Time')
            ax5.set_xlabel('Time (hours)')
            ax5.set_ylabel('Energy (J/kg)')
            ax5.legend()

            ax6 = fig.add_subplot(gs[2,2])
            ax6.plot(t_hours, angular_momentum, color='teal')
            ax6.set_title('Angular Momentum Magnitude')
            ax6.set_xlabel('Time (hours)')
            ax6.set_ylabel('h (m^2/s)')

            plt.tight_layout()
            plt.show()

            print(f"Initial altitude: {r_mag[0] - R_EARTH/1000:.1f} km")
            print(f"Initial velocity: {v_mag[0]:.3f} km/s")
            print(f"Initial eccentricity: {eccentricity[0]:.5f}")
            print(f"Final altitude: {r_mag[-1] - R_EARTH/1000:.1f} km")
            print(f"Final velocity: {v_mag[-1]:.3f} km/s")
            print(f"Final eccentricity: {eccentricity[-1]:.5f}")
            print(f"Initial angular momentum: {angular_momentum[0]:.3e} m^2/s")
            print(f"Final angular momentum: {angular_momentum[-1]:.3e} m^2/s")

    run_btn.on_click(on_click)

    ui = widgets.VBox([
        header,
        widgets.HBox([mass_sat, sim_days]),
        widgets.HTML("<b>Satellite Initial Position (km):</b>"),
        widgets.HBox([r_x, r_y, r_z]),
        widgets.HTML("<b>Satellite Initial Velocity (km/s):</b>"),
        widgets.HBox([v_x, v_y, v_z]),
        widgets.HBox([drag_toggle, j2_toggle, moon_toggle]),
        run_btn,
        out
    ])

    display(ui)

run_simulation_interface()






























